<a href="https://colab.research.google.com/github/MSBHUVANRAJ/medcuore_works/blob/main/Copy_of_lane_detection_dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Moddy2024/Lane-Detection.git


Cloning into 'Lane-Detection'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 69 (delta 22), reused 6 (delta 2), pack-reused 30
Receiving objects: 100% (69/69), 28.89 MiB | 2.30 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [2]:
from google.colab import drive
import os
# Mount Google Drive
drive.mount('/content/drive')

# Specify the path to the LANE_DETECTION folder in your Google Drive
drive_path = '/content/drive/MyDrive/LANE_DETECTION/'

# List files in the specified Google Drive folder
for filename in os.listdir(drive_path):
    print(os.path.join(drive_path, filename))


Mounted at /content/drive
/content/drive/MyDrive/LANE_DETECTION/full_CNN_train.p
/content/drive/MyDrive/LANE_DETECTION/full_CNN_labels.p
/content/drive/MyDrive/LANE_DETECTION/lane_modelpth.pth
/content/drive/MyDrive/LANE_DETECTION/lane_modeltar.tar
/content/drive/MyDrive/LANE_DETECTION/lanemodel.h5


In [3]:
!pip install keras_preprocessing
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 788.1 kB/s eta 0:00:00


In [4]:
import tensorflow as tf

In [5]:
import pickle
import numpy as np
# Load the training images
train_images_path = os.path.join(drive_path, 'full_CNN_train.p')
with open(train_images_path, 'rb') as file:
    train_images = pickle.load(file)

# Load the labels
labels_path = os.path.join(drive_path, 'full_CNN_labels.p')
with open(labels_path, 'rb') as file:
    labels = pickle.load(file)

# Make into arrays as the neural network wants these
train_images = np.array(train_images)
labels = np.array(labels)


In [6]:
# Normalize labels
labels= labels/255

train_images, labels = shuffle(train_images, labels)

X_train, X_val, y_train, y_val = train_test_split(train_images, labels, test_size=0.128)

# Batch size, epochs and pool size below are all paramaters to fiddle with for optimization
batch_size = 128
epochs = 25
pool_size = (2, 2)
input_shape = X_train.shape[1:]

In [7]:
train_datagen = ImageDataGenerator(channel_shift_range=0.2)
train_datagen.fit(X_train)

In [8]:
model=tf.keras.models.Sequential([
# Normalizes incoming inputs. First layer needs the input shape to work
tf.keras.layers.BatchNormalization(input_shape=input_shape),
tf.keras.layers.Conv2D(8, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv1'),
tf.keras.layers.Conv2D(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv2'),
# Pooling 1
tf.keras.layers.MaxPooling2D(pool_size=pool_size),

tf.keras.layers.Conv2D(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv3'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Conv2D(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv4'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Conv2D(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv5'),
tf.keras.layers.Dropout(0.2),
 # Pooling 2
tf.keras.layers.MaxPooling2D(pool_size=pool_size),

tf.keras.layers.Conv2D(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv6'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Conv2D(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv7'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Conv2D(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv8'),
# Pooling 3
tf.keras.layers.MaxPooling2D(pool_size=pool_size),
# Upsample 1
tf.keras.layers.UpSampling2D(size=pool_size),
#Deconv
tf.keras.layers.Conv2DTranspose(128, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv1'),
tf.keras.layers.Conv2DTranspose(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv2'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Conv2DTranspose(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv3'),
tf.keras.layers.Dropout(0.25),
# Upsample 2
tf.keras.layers.UpSampling2D(size=pool_size),

tf.keras.layers.Conv2DTranspose(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv4'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Conv2DTranspose(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv5'),
# Upsample 3
tf.keras.layers.UpSampling2D(size=pool_size),

tf.keras.layers.Conv2DTranspose(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv6'),
tf.keras.layers.Dropout(0.2),
tf.keras.layers.Conv2DTranspose(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv7'),
tf.keras.layers.Dropout(0.25),
tf.keras.layers.Conv2DTranspose(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv8'),
tf.keras.layers.Conv2DTranspose(1, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv9Finalone')
])

In [9]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.config.set_soft_device_placement(True)

Num GPUs Available:  1


In [10]:
# Compiling and training the model
model.compile(optimizer='Adam', loss='mean_squared_error',metrics=['accuracy'])

model.fit(train_datagen.flow(X_train, y_train, batch_size=batch_size),
                steps_per_epoch=len(X_train)/batch_size,
                epochs=epochs, verbose=1, validation_data=(X_val, y_val))

Epoch 1/25
86/86 [==============================] - 65s 455ms/step - loss: 0.0547 - accuracy: 0.9057 - val_loss: 0.0210 - val_accuracy: 0.9469
Epoch 2/25
86/86 [==============================] - 23s 264ms/step - loss: 0.0148 - accuracy: 0.9526 - val_loss: 0.0117 - val_accuracy: 0.9548
Epoch 3/25
86/86 [==============================] - 23s 258ms/step - loss: 0.0105 - accuracy: 0.9559 - val_loss: 0.0090 - val_accuracy: 0.9567
Epoch 4/25
86/86 [==============================] - 23s 259ms/step - loss: 0.0093 - accuracy: 0.9568 - val_loss: 0.0083 - val_accuracy: 0.9571
Epoch 5/25
86/86 [==============================] - 23s 268ms/step - loss: 0.0086 - accuracy: 0.9572 - val_loss: 0.0075 - val_accuracy: 0.9576
Epoch 6/25
86/86 [==============================] - 23s 259ms/step - loss: 0.0081 - accuracy: 0.9575 - val_loss: 0.0073 - val_accuracy: 0.9577
Epoch 7/25
86/86 [==============================] - 23s 263ms/step - loss: 0.0078 - accuracy: 0.9577 - val_loss: 0.0072 - val_accuracy: 0.9581

In [11]:
model.trainable = False
model.save('/content/drive/MyDrive/LANE_DETECTION/lanemodel.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
model.save('/content/drive/MyDrive/LANE_DETECTION/lane_modelpth.pth')

In [13]:
model.save('/content/drive/MyDrive/LANE_DETECTION/lane_modeltar.tar')

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (Batch  (None, 80, 160, 3)        12        
 Normalization)                                                  
                                                                 
 Conv1 (Conv2D)              (None, 78, 158, 8)        224       
                                                                 
 Conv2 (Conv2D)              (None, 76, 156, 16)       1168      
                                                                 
 max_pooling2d (MaxPooling2  (None, 38, 78, 16)        0         
 D)                                                              
                                                                 
 Conv3 (Conv2D)              (None, 36, 76, 16)        2320      
                                                                 
 dropout (Dropout)           (None, 36, 76, 16)        0

In [15]:
!pip install scipy

In [16]:
import numpy as np
import cv2
from scipy.ndimage import zoom
from moviepy.editor import VideoFileClip

from keras.models import load_model
import matplotlib.pyplot as plt


In [34]:

class Lanes():
    def __init__(self):
        self.recent_fit = []
        self.avg_fit = []

# def road_lines(image):
#     """ Takes in a road image, re-sizes for the model,
#     predicts the lane to be drawn from the model in G color,
#     recreates an RGB image of a lane and merges with the
#     original road image.
#     """
#     image_shape = image.shape
#     # Get image ready for feeding into the model
#     small_img = cv2.resize(image, (160, 80))
#     small_img = np.array(small_img)
#     small_img = small_img[None, :, :, :]

#     # Make prediction with neural network (un-normalize value by multiplying by 255)
#     prediction = model.predict(small_img)[0] * 255
#     # Add lane prediction to the list for averaging
#     lanes.recent_fit.append(prediction)

#     # Only using the last five for average
#     if len(lanes.recent_fit) > 5:
#         lanes.recent_fit = lanes.recent_fit[1:]

#     # Calculate the average detection
#     lanes.avg_fit = np.mean(np.array([i for i in lanes.recent_fit]), axis=0)

#     # Generate fake R & B color dimensions, stack with G
#     blanks = np.zeros_like(lanes.avg_fit).astype(np.uint8)

#     # for setting different color for the detection
#     lane_drawn = np.dstack((lanes.avg_fit, lanes.avg_fit, blanks))

#     # Re-size to match the original image
#     lane_image = cv2.resize(lane_drawn, (image_shape[1], image_shape[0]))

#     # Merge the lane drawing onto the original image
#     result = cv2.addWeighted(image, 1, lane_image, 1, 0, dtype=image.dtype)

#     return result

# lanes = Lanes()
def road_lines(image):
    """ Takes in a road image, re-sizes for the model,
    predicts the lane to be drawn from the model in G color,
    recreates an RGB image of a lane and merges with the
    original road image.
    """
    image_shape = image.shape
    # Get image ready for feeding into the model
    small_img = cv2.resize(image, (160, 80))
    small_img = np.array(small_img)
    small_img = small_img[None, :, :, :]

    # Make prediction with neural network (un-normalize value by multiplying by 255)
    prediction = model.predict(small_img)[0] * 255
    # Add lane prediction to the list for averaging
    lanes.recent_fit.append(prediction)

    # Only using the last five for average
    if len(lanes.recent_fit) > 5:
        lanes.recent_fit = lanes.recent_fit[1:]

    # Calculate the average detection
    lanes.avg_fit = np.mean(np.array([i for i in lanes.recent_fit]), axis=0)

    # Generate fake R & B color dimensions, stack with G
    blanks = np.zeros_like(lanes.avg_fit).astype(np.uint8)

    # for setting different color for the detection
    lane_drawn = np.dstack((lanes.avg_fit, lanes.avg_fit, blanks))

    # Re-size to match the original image
    lane_image = cv2.resize(lane_drawn, (image_shape[1], image_shape[0]))

    # Merge the lane drawing onto the original image
    result = cv2.addWeighted(image.astype(np.uint8), 1, lane_image.astype(np.uint8), 1, 0)

    return result
lanes = Lanes()
# class Lanes():
#     def __init__(self):
#         self.recent_fit = []
#         self.avg_fit = []

# def road_lines(image):
#     """ Takes in a road image, re-sizes for the model,
#     predicts the lane to be drawn from the model in G color,
#     recreates an RGB image of a lane and merges with the
#     original road image.
#     """
#     image_shape = image.shape
#     # Get image ready for feeding into model
#     small_img = cv2.resize(image, (160, 80))
#     small_img = np.array(small_img)
#     small_img = small_img[None, :, :, :]

#     # Make prediction with neural network (un-normalize value by multiplying by 255)
#     prediction = model.predict(small_img)[0] * 255
#     # Add lane prediction to list for averaging
#     lanes.recent_fit.append(prediction)

#     # Only using last five for average
#     if len(lanes.recent_fit) > 5:
#         lanes.recent_fit = lanes.recent_fit[1:]

#     # Calculate average detection
#     lanes.avg_fit = np.mean(np.array([i for i in lanes.recent_fit]), axis=0)

#     # Generate fake R & B color dimensions, stack with G
#     blanks = np.zeros_like(lanes.avg_fit).astype(np.uint8)

#     # for setting different color for the detection
#     lane_drawn = np.dstack((lanes.avg_fit, lanes.avg_fit, blanks))

#     # Re-size to match the original image
#     lane_image = cv2.resize(lane_drawn, (image_shape,image_shape))

#     # Merge the lane drawing onto the original image
#     result = cv2.addWeighted(image.astype(np.uint8), 1, lane_image.astype(np.uint8), 1, 0)

#     return result

# lanes = Lanes()


In [39]:


# Where to save the output video
vid_output = '/content/drive/MyDrive/output_videos/7.mp4'

clip1 = VideoFileClip("/content/drive/MyDrive/output_videos/no_lane.mp4")

vid_clip = clip1.fl_image(road_lines)
vid_clip.write_videofile(vid_output, audio=False)

1/1 [==============================] - 0s 26ms/step
Moviepy - Building video /content/drive/MyDrive/output_videos/7.mp4.
Moviepy - Writing video /content/drive/MyDrive/output_videos/7.mp4



t:   0%|          | 0/1304 [00:00<?, ?it/s, now=None]

1/1 [==============================] - 0s 29ms/step


t:   0%|          | 3/1304 [00:00<01:30, 14.35it/s, now=None]

1/1 [==============================] - 0s 37ms/step


...

t: 100%|█████████▉| 1303/1304 [02:26<00:00,  9.01it/s, now=None]

1/1 [==============================] - 0s 27ms/step


t: 100%|██████████| 1304/1304 [02:27<00:00,  8.76it/s, now=None]

1/1 [==============================] - 0s 21ms/step


Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/output_videos/7.mp4
